# ДЗ №1

**Описание сервиса**. В качестве учебного проекта предлагается простая регрессионная модель прогнозирования стоимости объектов недвижимости в Санк-Петербурге. Пользователь вводит основные параметры желаемого объекта недвижимости и получает примерную стоимость данного объекта.

In [30]:
from dataclasses import dataclass, field
from typing import List, Set
import re
import pickle
from datetime import datetime
from pathlib import Path

## Базовые сущности

In [ ]:
@dataclass
class User:

    """Класс для представления пользователя в системе
    
    Attributes:
        id (int): уникальный идентификатор пользователя
        username (str): логин пользователя
        existing_usernames (set): множество уникальных логинов пользователей
        password (str): пароль пользователя
        email (str): email пользователя
        balance (float): счет пользователя
        transactions (List[Transactions]): список транзакций пользователя
        requests (List[Predictions]): список запросов пользователя
      
    """
    
    id: int
    username: str
    password: str
    email: str
    balance: float
    transactions: List['Transaction'] = field(default_factory=list)
    predictions: List['Prediction'] = field(default_factory=list)

    __existing_usernames: Set[str] = field(default_factory=set, init=False)
    

    def __post_init__(self) -> None:
        self._validate_username()
        self._validate_email()
        self._validate_password()

    def _validate_username(self) -> None:
        """Проверяет уникальность введенного логина при регистрации."""
        if self.username in self.__existing_usernames:
            raise ValueError(f"Username '{self.username}' already exists")
        # Сохраняем уникальный логин
        self.__existing_usernames.add(self.username)

    def _validate_email(self) -> None:
        """Проверяет корректность email."""
        email_pattern = re.compile(r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$')
        if not email_pattern.match(self.email):
            raise ValueError("Invalid email format")

    def _validate_password(self) -> None:
        """Проверяет минимальную длину пароля."""
        if len(self.password) < 8:
            raise ValueError("Password must be at least 8 characters long")

    def view_transactions(self) -> List['Transaction']:
        """Вывод всех транзакций."""
        return self.transactions

    def view_predictions(self) -> List['Prediction']:
        """Вывод всех предсказаний."""
        return self.predictions
    
    def get_balance(self) -> float:
        """Вывод баланса пользователя"""
        return self.balance
    
    def add_balance(self, amount: float) -> None:
        """Пополнение баланса пользователем"""
        self.balance += amount
                
    def deduct_balance(self, amount: float) -> None:
        """Списание с баланса пользователя"""
        if self.balance >= amount:
            self.balance -= amount
        else:
            raise ValueError("Insufficient funds. Top up your balance and try again")
    

In [ ]:
@dataclass
class Administrator(User):

    """Класс для представления администратора. Наследуется из класса User"""

    def add_balance_to_user(self, user: User, amount: float) -> None:
        """Пополнение счета выбранного пользователя"""
        user.add_balance(amount)
        
    def view_all_transactions(self, user: User) -> list:
        """Просмотр транзакций выбранного пользователя"""
        return user.transactions
    
    def view_all_predictions(self, user: User) -> list:
        """Просмотр предсказаний модели для выбранного пользователя"""
        return user.predictions

In [ ]:
@dataclass
class Model:
    """Класс для представления модели,
    включая проверку входных данных.
    
    Атрибуты:
        param_1 (int): параметр 1
        param_2 (str): категориальная переменная 
        param_3 (float): параметр 3
        result (float): ответ модели на запрос
                    
    """
    param_1: int
    param_2: str
    param_3: float
    model_path: str
    input_data: dict
    result: float = field(init=False)
    

    def __post_init__(self):
        """Инициализация класса с проверкой входных данных."""
        self.model = self.load_model()
        self.input_data = input_data()
        self.param_1 = self.validate_param_1(input_data.get('param_1'), 20, 100)
        self.param_2 = self.validate_param_2(input_data.get('param_2'))
        self.param_3 = self.validate_param_3(input_data.get('param_3'), 0.0, 10.0)
        self.result = self.get_result()

    def validate_param_1(self, param_1: int, limit_1a: int, limit_1b: int) -> int:
        """Проверка валидности первого параметра."""
        if param_1 is None or not (limit_1a <= param_1 <= limit_1b):
            raise ValueError(f"Enter value between {limit_1a} and {limit_1b}.")
        return param_1

    def validate_param_2(self, param_2: str) -> str:
        """Проверка валидности второго параметра."""
        categories = ['cat_1', 'cat_2', 'cat_3']
        if param_2 is None or param_2 not in categories:
            raise ValueError("Param_2 must be cat_1, cat_2 ot cat_3.")
        return param_2
    
    def validate_param_3(self, param_3: float, limit_3a: float, limit_3b: float) -> float:
        """Проверка валидности третьего параметра."""
        if param_3 is None or not (limit_3a <= param_3 <= limit_3b):
            raise ValueError(f"Enter value between {limit_3a} and {limit_3b}.")
        return param_3
    
    def load_model(self):
        """Загрузка модели из файла"""
        with open(self.model_path, 'rb') as f:
            return pickle.load(f)
    
    def get_result(self) -> float:
        """Расчет результата на основании параметров и весов модели."""
        result = self.model.predict([list(self.input_data.values())])[0]
        return result

In [ ]:
@dataclass
class Prediction:
    """Класс для сохранения результатов работы модели.
    
    Атрибуты:
        user_id (int): пользователь, в историю которого сохраняется предсказание
        task_id (int): номер запроса
        result (float): результат работы модели
        cost (float): стоимость
        timestamp (datetime): метка времени, когда был сделан запрос
            
    """

    user_id: int
    task_id: int
    result: float
    cost: float
    timestamp: datetime = field(default_factory=datetime.now)
    
    def to_dict(self):
        return {
            'user_id': self.user_id,
            'task_id': self.task_id,
            'result': self.result,
            'cost': self.cost,
            'timestamp': self.timestamp
        }

In [34]:
@dataclass
class PredictionTask:

    """Класс для выполнения предсказаний
    
    Атрибуты:
        user_id (int): пользователь, сделавший запрос на прогноз
        task_id (int): идентификатор задачи
        model (class): обращение к классу Model
        input_data (dict): словарь с входными данными
    
    """

    task_id: int
    user: User
    model: Model
    prediction: Prediction
    input_data: dict
    timestamp: datetime = field(default_factory=datetime.now)

    def execute(self) -> 'Prediction':
        """Выполнение запроса на получение прогноза"""
        if self.user.balance < self.prediction.cost:
            raise ValueError("Insufficient balance")
        self.user.deduct_balance(self.prediction.cost)
        result = self.model.get_result(self.input_data)
        return Prediction(
            user_id=self.user.id,
            task_id=self.task_id,
            result=result,
            cost=self.prediction.cost,
            timestamp=self.timestamp
        )

In [35]:
@dataclass
class Transaction:
    """Класс для сохранения операций со счетом.
    
    Атрибуты:
        user_id (int): пользователь, в историю которого сохраняется транзакция
        timestamp (datetime): метка времени, когда был сделан запрос
        description (str): описание транзакции
        amount (float): сумма операции
        transaction_type (str): тип транзакции
        transactions (list): список для хранения транзакций
    
    """

    user_id: int
    description: str
    amount: float
    transaction_type: str
    timestamp: datetime = field(default_factory=datetime.now)
    transactions: List['Transaction'] = field(default_factory=list)

    def __post_init__(self):
        if self.transactions is None:
            self.transactions = []

    def to_dict(self):
        return {
            'user_id': self.user_id,
            'description': self.description,
            'amount': self.amount,
            'transaction_type': self.transaction_type,
            'timestamp': self.timestamp
        }